In [88]:
## Catriona Miller - 17/06/2022
# This script can be made much more efficient if I use a better finding algorithm than a double for loop 
# (e.g. change the snps to a sorted integer list and use a smarter algorithm than checking every entry). 
# However, for now it is good enough as it doesn't take too long to run (~1-2 minutes) and I wanted something quick to code.

import pandas as pd
import os
# These variables should be changed depending on what you're running. Nothing else needs to be updated

# The reference GRN where you are searching for regulation (beta) (on figshare. See manuscript for details on adult and fetal GRN.)
inputGRN = 'significant_eqtls_fetal_brain_cortex.txt'
# The file contains snps and genes (output of m3d; same as supplementary tables S4 and S5)
inputData = 'level4_sig_interactions.txt'
# Where you want to output the updated file with the regulation column
outputData = 'level4_sig_interactions_fetal_reg.txt'

outputGene = 'gene_position_level4_fetal.txt'

In [89]:
data = pd.read_csv(inputGRN,sep ="\t")
snps = pd.DataFrame(data)

m3d_data = pd.read_csv(inputData,sep="\t")
m3d = pd.DataFrame(m3d_data)

C:\Users\catri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5,6,7,9,10,12,16,17,18,21,23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [90]:
snps.head()

,snp,gencode_id,gene,tissue,adj_pval,beta,beta_se,maf,interaction_type,hic_score,...,gene_start,gene_end,distance,cell_lines,cell_line_hic_scores,expression,max_expressed_tissue,max_expression,min_expressed_tissue,min_expression
0,rs60914341,ENSG00000121310.16,ECHDC2,Fetal_Brain_Cortex_Walker2019,0.035413,0.223574,0.049243,0.24424,Trans-intrachromosomal,1.67,...,52895910,52927212,1947265.0,"Germinal_zone_neurons_Won2016, Cortical_plate_...","1.0, 0.67",3.94494,Fetal_Brain_Cortex_Walker2019,3.94494,Fetal_Brain_Cortex_Walker2019,3.94494
1,rs61782124,ENSG00000121310.16,ECHDC2,Fetal_Brain_Cortex_Walker2019,0.048041,0.275777,0.061964,0.191244,Trans-intrachromosomal,1.67,...,52895910,52927212,1342346.0,"Cortical_plate_neurons_Won2016, Germinal_zone_...","1.0, 0.67",3.94494,Fetal_Brain_Cortex_Walker2019,3.94494,Fetal_Brain_Cortex_Walker2019,3.94494
2,rs35375774,ENSG00000121310.16,ECHDC2,Fetal_Brain_Cortex_Walker2019,0.004451,0.308079,0.060448,0.195853,Trans-intrachromosomal,0.67,...,52895910,52927212,1374106.0,"Germinal_zone_neurons_Won2016, Cortical_plate_...","0.33, 0.33",3.94494,Fetal_Brain_Cortex_Walker2019,3.94494,Fetal_Brain_Cortex_Walker2019,3.94494
3,rs34886807,ENSG00000121310.16,ECHDC2,Fetal_Brain_Cortex_Walker2019,0.004451,0.308079,0.060448,0.195853,Trans-intrachromosomal,0.67,...,52895910,52927212,1373573.0,"Germinal_zone_neurons_Won2016, Cortical_plate_...","0.33, 0.33",3.94494,Fetal_Brain_Cortex_Walker2019,3.94494,Fetal_Brain_Cortex_Walker2019,3.94494
4,rs61782122,ENSG00000121310.16,ECHDC2,Fetal_Brain_Cortex_Walker2019,0.008007,0.303015,0.061257,0.182028,Trans-intrachromosomal,0.67,...,52895910,52927212,1372543.0,"Germinal_zone_neurons_Won2016, Cortical_plate_...","0.33, 0.33",3.94494,Fetal_Brain_Cortex_Walker2019,3.94494,Fetal_Brain_Cortex_Walker2019,3.94494


In [91]:
m3d.head()

,snp,trait,gene,eqtl_type,chromq,posq,rsidq,chromt,post,rsidt,corr,dprime
0,rs910049,Hepatitis C induced liver cirrhosis,HLA-DRB1,spatial,6.0,32316388.0,rs3117123,6.0,32315726.0,rs910049,1.0,1.0
1,rs910049,Hepatitis C induced liver cirrhosis,HLA-DRB1,spatial,6.0,32315506.0,rs3117127,6.0,32315726.0,rs910049,1.0,1.0
2,rs910049,Hepatitis C induced liver cirrhosis,HLA-DRB1,spatial,6.0,32315100.0,rs3117129,6.0,32315726.0,rs910049,1.0,1.0
3,rs910049,Hepatitis C induced liver cirrhosis,HLA-DRB1,spatial,6.0,32312081.0,rs6904320,6.0,32315726.0,rs910049,1.0,1.0
4,rs910049,Hepatitis C induced liver cirrhosis,HLA-DRB1,spatial,6.0,32312352.0,rs7383144,6.0,32315726.0,rs910049,1.0,1.0


In [92]:
regulation = []
genestart = []
geneend = []
genechr = []
typ = []
i=0
for snp in m3d['rsidq']:
    j=0
    for reference in snps['snp']:
        if snp == reference:
            if m3d['gene'][i] == snps['gene'][j]:
                reg = snps['beta'][j]
                start = snps['gene_start'][j]
                end = snps['gene_end'][j]
                chrom = snps['gene_chr'][j]
                inter_typ = snps['interaction_type'][j]
                regulation.append(reg)
                genestart.append(start)
                geneend.append(end)
                genechr.append(chrom)
                typ.append(inter_typ)
        j=j+1
    i=i+1

m3d['regulation'] = regulation
m3d['genestart'] = genestart
m3d['geneend'] = geneend
m3d['genechr'] = genechr
m3d['interaction_type'] = typ

In [93]:
m3d.to_csv(outputData,sep="\t",index=None)

In [94]:
# Rearranging the dataframe so that it is suitable input for ChromoMap
genepos = m3d.drop(columns=['trait','eqtl_type','snp','interaction_type'])
order = ['gene','genechr','genestart','geneend','regulation']
genepos = genepos[order]
genepos.head()

,gene,genechr,genestart,geneend,regulation
0,HLA-DRB1,chr6,32578769,32589848,-0.449394
1,HLA-DRB1,chr6,32578769,32589848,-0.449394
2,HLA-DRB1,chr6,32578769,32589848,-0.449394
3,HLA-DRB1,chr6,32578769,32589848,-0.450296
4,HLA-DRB1,chr6,32578769,32589848,-0.449394


In [95]:
#TURN ON OR OFF IF REQUIRED
genepos = genepos.drop(columns=['regulation'])
genepos = genepos.drop_duplicates()
genepos.head()


,gene,genechr,genestart,geneend
0,HLA-DRB1,chr6,32578769,32589848
78,TRAF1,chr9,120902393,120929173
103,IQGAP1,chr15,90388218,90502243
191,DHX33,chr17,5440912,5469060
277,BAZ1A,chr14,34752731,34862322


In [96]:
genepos.to_csv(outputGene,sep="\t",index=None,header=None)